In [ ]:
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pylab as pl
import matplotlib.gridspec as gridspec
import scipy.ndimage as ndimage
from matplotlib.ticker import AutoMinorLocator, FormatStrFormatter
import matplotlib.mlab as mlab
from matplotlib import cm
import pylab
Pfit = np.polynomial.Polynomial.fit

mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['axes.linewidth'] = 1

mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['axes.labelweight'] = 'bold'

plt.rc('text', usetex=True) ## this line is necessary to use the correct fonts 
plt.rc('font', family='serif') # this line is necessary to use the correct font'

def plot_prop(f1,ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    # Only show ticks on the left and bottom spines
    ax.yaxis.set_ticks_position('left') 
    ax.xaxis.set_ticks_position('bottom')
    ## put some space betweent the axes
    #ax.spines['bottom'].set_visible(False)
    #ax.get_xaxis().set_visible(False)
    ax.spines['left'].set_position(('axes', -0.02))
    ax.spines['bottom'].set_position(('axes', -0.02))
    ax.xaxis.set_minor_locator(AutoMinorLocator(1))
    ax.yaxis.set_minor_locator(AutoMinorLocator(1))

    # x and y tickers
    plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%g'))
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%g'))
    plt.tick_params(direction='out',which='major', length=5,width=1,colors='k',labelsize=f1)
    
    plt.tick_params(direction='out',which='minor',length=4,width=1, color='k',axis='x')
    plt.tick_params(direction='out',which='minor',length=0.1,width=1, color='k',axis='y')
    
    return;



#path1='/Users/khaledi/Google Drive/7_Sherlock_codes/Sherlock1/26_Tinitus/7_sigmad_Astim_M/'
rall1=np.loadtxt('all2.dat')
lena1=len(rall1)

M_el=np.unique(rall1[:,2]); lenm=len(M_el)
Astim=np.unique(rall1[:,3]); lenA=len(Astim)
sigmad=np.unique(rall1[:,4]); lens=len(sigmad)

r1=np.zeros([lens,lenm,lenA])
A1=np.zeros([lens,lenm,lenA])
r_opt=np.zeros([lens,lenm])
A_opt=np.zeros([lens,lenm])
Gap_index=np.zeros([lens,lenm])
overalp=np.zeros([lens,lenm])
Fk=np.zeros([lens,lenm])
Ak=np.zeros([lens,lenm])
Ak1=np.zeros([lens,lenm])
Ls=10
tf=rall1[19,0]
all_sz=len(Astim)*len(sigmad)*len(M_el)+1
all_data=np.zeros([4,all_sz])

In [ ]:
Astim_fit=np.linspace(min(Astim),max(Astim),100)
sigmad_fit=np.linspace(min(sigmad),max(sigmad),103)
M_fit=np.linspace(min(M_el),max(M_el),49)
#np.savetxt('all_simualtion.dat',all_data)

In [ ]:

for i1 in range(0,lena1):
    if (rall1[i1,0]==tf):
        s1=rall1[i1,4]
        M1=rall1[i1,2]
        A1=rall1[i1,3]
        i=np.where(sigmad==s1)[0][0]
        j=np.where(M_el==M1)[0][0]
        k=np.where(Astim==A1)[0][0]
        r1[i,j,k]=sum(rall1[i1-19:i1,5])/19
        #r1[i,j,k]=rall1[i1,5]

        
k1=-1
for i in range(0,lens):
    for j in range(0,lenm):
        for k in range(0,lenA):
            k1=k1+1
            all_data[0,k1]=s1
            all_data[1,k1]=M1
            all_data[2,k1]=A1
            all_data[3,k1]=r1[i,j,k]
        
        
for i in range(0,lens):
    for j in range(0,lenm):
        r_opt[i,j]=np.min(r1[i,j,:])
        index_array = np.argmin(r1[i,j,:])
        A_opt[i,j]=Astim[index_array]
        Fk[i,j]=(Ls/ M_el[j])- Astim[index_array]*np.pi*sigmad[i]
        Ak[i,j]=sigmad[i]*M_el[j]
        Ak1[i,j]=sigmad[i]*M_el[j]*Astim[index_array]
        if ( Fk[i,j]>0): 
            Gap_index[i,j]= (M_el[j]-1) * (Ls/M_el[j]  
                                           -Fk[i,j]) / (Ls*(1-1/M_el[j] + Astim[index_array]*np.pi*sigmad[i]))


In [ ]:
#A_opt[A_opt>0.8]=1
f1=20
fig=plt.figure()
fig.set_size_inches(12,4)  

ax=plt.subplot(121)
plot_prop(f1,ax)
C1=plt.contourf(sigmad,M_el,np.transpose(r_opt),100,cmap='nipy_spectral')
for c in C1.collections:c.set_edgecolor("face") # this removes the white lines between the regions 
t=[0.1,0.3,0.5,0.7]
plt.ylabel(r'$N_s$(electrodes)',fontsize=f1)
plt.xlabel(r'$\sigma_d$',fontsize=f1)
plt.text(0.02,51,r'(a) $ \langle \rho_\mathrm{opt} \rangle $',fontsize=f1)
plt.yticks([1,10,20,30,40,50])
cb=plt.colorbar(C1,shrink=1.,pad=0.02,ticks=t)
cb.ax.tick_params(labelsize=f1-4) 
plt.xscale('log')


ax=plt.subplot(122)
plot_prop(f1,ax)
C1=plt.contourf(sigmad,M_el,np.log10(np.transpose(A_opt)),100,cmap='nipy_spectral')
#C1=plt.contourf(sigmad,M_el,np.transpose(A_opt),100,cmap='nipy_spectral')
for c in C1.collections:c.set_edgecolor("face") # this removes the white lines between the regions 

plt.yticks([1,10,20,30,40,50])
t=[np.log10(0.2),np.log10(1),np.log10(10),np.log10(100)]
cb=plt.colorbar(C1,shrink=1.,pad=0.02,ticks=t)
cb.ax.set_yticklabels([r'$0.2$', '$1$','$10$' ,'$> 100$'])
plt.text(0.02,51,r'(b) $ I_\mathrm{opt} $',fontsize=f1)
plt.xticks([0.01,0.1,1])
cb.ax.tick_params(labelsize=f1-4) 
plt.tight_layout(pad=0.2)
plt.xlabel(r'$\sigma_d$',fontsize=f1)
plt.xscale('log')

plt.tight_layout(pad=0.8)

#plt.savefig('4_R_opt_Astim_opt.pdf',dpi=1000)
#plt.savefig('4_R_opt_Astim_opt.png',dpi=500)
#plt.show()

In [ ]:
f1=20
fig=plt.figure()
fig.set_size_inches(12,4)  

ax=plt.subplot(121)
plot_prop(f1,ax)
C1=plt.contourf(sigmad,M_el,np.log10(np.transpose(Ak)),100,cmap='nipy_spectral')
for c in C1.collections:c.set_edgecolor("face") # this removes the white lines between the regions 
    
plt.text(0.02,51,r'(a) $ N_s \sigma_d $',fontsize=f1)
plt.ylabel(r'$N_s$(electrodes)',fontsize=f1)
plt.xlabel(r'$\sigma_d$',fontsize=f1)
t=[np.log10(0.02),np.log10(0.1),np.log10(1),np.log10(10),np.log10(100)]
cb=plt.colorbar(C1,shrink=1.,pad=0.02,ticks=t)
plt.yticks([1,10,20,30,40,50])
cb.ax.set_yticklabels([r'$0.02$', '$0.1$','$1$','$10$' ,'$100$'],fontsize=f1)
plt.xscale('log')  

ax=plt.subplot(122)
plot_prop(f1,ax)
C1=plt.contourf(sigmad,M_el,np.log10(np.transpose(Ak1)),100,cmap='nipy_spectral')
for c in C1.collections:c.set_edgecolor("face") # this removes the white lines between the regions 
#plt.colorbar()
plt.text(0.02,51,r'(b) $ I_\mathrm{opt} N_s \sigma_d $',fontsize=f1)
plt.xlabel(r'$\sigma_d$',fontsize=f1)
plt.yticks([1,10,20,30,40,50])
t=[-.22,0.22,0.66,1.1,1.42]
#t=[np.log10(0.1),np.log10(0.02),np.log10(1),np.log10(10),np.log10(100)]
cb=plt.colorbar(C1,shrink=1.,pad=0.02,ticks=t)
cb.ax.set_yticklabels([r'$0.6$', '$1.67$','$4.57$','$12.6$' ,'$26.3$'],fontsize=f1)
plt.xscale('log') 
plt.tight_layout(pad=0.8)

#plt.savefig('5_Ns_sgimad.pdf',dpi=1000)
#plt.savefig('5_Ns_sigmad.png',dpi=500)

plt.show()
    

In [ ]:
f1=20
fig=plt.figure()
fig.set_size_inches(12,4)  

ax=plt.subplot(121)
plot_prop(f1,ax)
C1=plt.contourf(sigmad,M_el,np.transpose(r_opt),100,cmap='nipy_spectral')
for c in C1.collections:c.set_edgecolor("face") # this removes the white lines between the regions 
t=[0.1,0.3,0.5,0.7]
plt.ylabel(r'$N_s$(electrodes)',fontsize=f1)
plt.xlabel(r'$\sigma_d$',fontsize=f1)
plt.text(0.02,51,r'(a) $ \langle \rho_\mathrm{opt} \rangle $',fontsize=f1)
plt.yticks([1,10,20,30,40,50])
cb=plt.colorbar(C1,shrink=1.,pad=0.02,ticks=t)
cb.ax.tick_params(labelsize=f1-4) 
plt.xscale('log')

ax=plt.subplot(122)
plot_prop(f1,ax)
C1=plt.contourf(sigmad,M_el,np.transpose(Gap_index),100,cmap='nipy_spectral')
plt.yticks([1,10,20,30,40,50])
t=[0.1,0.3,0.5,0.7,.9,1.1,1.3,1.5]
cb=plt.colorbar(C1,shrink=1.,pad=0.02)#,ticks=t)
plt.text(0.02,51,r'(b) $ g_\mathrm{k}(Ns,I_\mathrm{opt},\sigma_d) $',fontsize=f1)

plt.xticks([0.01,0.1,1])
cb.ax.tick_params(labelsize=f1-4) 
plt.tight_layout(pad=0.2)
plt.xlabel(r'$\sigma_d$',fontsize=f1)
plt.xscale('log')
plt.tight_layout(pad=0.8)

#plt.savefig('8_R_g_Ns_sgimad_L10.pdf',dpi=1000)
#plt.savefig('8_R_g_Ns_sigmad_L10.png',dpi=500)

plt.show()

In [ ]:
#A_opt[A_opt>0.8]=1
f1=20
fig=plt.figure()
fig.set_size_inches(12,4)  

ax=plt.subplot(121)
plot_prop(f1,ax)
C1=plt.contourf(sigmad,M_el,np.transpose(r_opt),100,cmap='nipy_spectral')
for c in C1.collections:c.set_edgecolor("face") # this removes the white lines between the regions 
t=[0.1,0.3,0.5,0.7]
plt.ylabel(r'$N_s$(electrodes)',fontsize=f1)
plt.xlabel(r'$\sigma_d$',fontsize=f1)
plt.text(0.02,51,r'(a) $ \langle \rho_\mathrm{opt} \rangle $',fontsize=f1)
plt.yticks([1,10,20,30,40,50])
cb=plt.colorbar(C1,shrink=1.,pad=0.02,ticks=t)
cb.ax.tick_params(labelsize=f1-4) 
plt.xscale('log')


ax=plt.subplot(122)
plot_prop(f1,ax)
C1=plt.contourf(sigmad,M_el,np.log10(np.transpose(A_opt)),100,cmap='nipy_spectral')
#C1=plt.contourf(sigmad,M_el,np.transpose(A_opt),100,cmap='nipy_spectral')
for c in C1.collections:c.set_edgecolor("face") # this removes the white lines between the regions 

plt.yticks([1,10,20,30,40,50])
t=[np.log10(0.2),np.log10(1),np.log10(10),np.log10(100)]
cb=plt.colorbar(C1,shrink=1.,pad=0.02,ticks=t)
cb.ax.set_yticklabels([r'$0.2$', '$1$','$10$' ,'$> 100$'])
plt.text(0.02,51,r'(b) $ I_\mathrm{opt} $',fontsize=f1)
plt.xticks([0.01,0.1,1])
cb.ax.tick_params(labelsize=f1-4) 
plt.tight_layout(pad=0.2)
plt.xlabel(r'$\sigma_d$',fontsize=f1)
plt.xscale('log')

plt.tight_layout(pad=0.8)

#plt.savefig('4_R_opt_Astim_opt.pdf',dpi=1000)
#plt.savefig('4_R_opt_Astim_opt.png',dpi=500)
#plt.show()

In [ ]:
f1=20
fig=plt.figure()
fig.set_size_inches(12,4)  

ax=plt.subplot(121)
plot_prop(f1,ax)

plt.plot(Astim,r1[1,1,:],'b--s',label='$\sigma_d=0.02$')
plt.plot(Astim,r1[5,1,:],'r--o',label='$\sigma_d=0.06$')
plt.plot(Astim,r1[11,1,:],'m--*',label='$\sigma_d=0.3$')
plt.title(r'(a)~~$N_s=4$',fontsize=f1)
plt.ylim(0,1)
plt.legend(frameon=False,fontsize=f1-8,loc=1)
plt.xlabel('stimulation strength',fontsize=f1)
plt.ylabel(r'order parameter',fontsize=f1)
plt.xscale('log')

ax=plt.subplot(122)
plot_prop(f1,ax)

j=9
plt.plot(Astim,r1[j,0,:],'k--s',label='$N_s=2$')
plt.plot(Astim,r1[j,2,:],'g--o',label='$N_s=6$')
plt.plot(Astim,r1[j,10,:],'C1--*',label='$N_s=22$')
plt.title(r'(b)~~$\sigma_d=0.9$',fontsize=f1)
plt.ylim(0,1)
plt.legend(frameon=False,fontsize=f1-5,loc=1)
plt.xlabel('stimulation strength',fontsize=f1)
plt.xscale('log')
plt.tight_layout(pad=0.8)
plt.savefig('7_Astim_rho.pdf',dpi=1000)
plt.savefig('7_Astim_rho.png',dpi=500)
plt.show()

In [ ]:
f1=20
fig=plt.figure()
fig.set_size_inches(12,4)  

ax=plt.subplot(121)
plot_prop(f1,ax)

plt.plot(M_el,r_opt[2,:],'b--s',label='$\sigma_d=0.03$')
plt.plot(M_el,r_opt[10,:],'r--o',label='$\sigma_d=0.2$')
plt.plot(M_el,r_opt[20,:],'m--*',label='$\sigma_d=1.2$')
plt.xlim(2,50)
plt.ylim(0,0.8)
plt.legend(frameon=False,fontsize=f1,loc=1)
plt.title(r'(a)~~$\rho_\mathrm{opt}$ vs $N_s$',fontsize=f1)
plt.ylabel(r'$ \langle \rho_\mathrm{opt} \rangle $',fontsize=f1)
plt.xlabel(r'$N_s$',fontsize=f1)

ax=plt.subplot(122)
plot_prop(f1,ax)

plt.plot(sigmad,r_opt[:,0],'b--s',label='$N_s=2$')
plt.plot(sigmad,r_opt[:,1],'r--o',label='$N_s=4$')
plt.plot(sigmad,r_opt[:,10],'m--*',label='$N_s=22$')
plt.ylim(0,0.8)
plt.xscale('log')
plt.xlabel(r'$\sigma_d$',fontsize=f1)
plt.legend(frameon=False,fontsize=f1,loc=0)
plt.title(r'(b)~~$\rho_\mathrm{opt}$ vs $\sigma_d$',fontsize=f1)
plt.tight_layout(pad=0.8)
plt.savefig('9_Ropt_Ns_sigmad_2D.pdf',dpi=1000)
plt.savefig('9_Ropt_Ns_sigmad_2D.png',dpi=500)
plt.show()

In [ ]:
M_el[10]